# Opening
This is my exercise to explore a dataset based on courses I've learned for few months. I'm beginner on Data Science and I have strong passionate to learn it! Your suggestions and advices is important for me!

I'm only work in dataset train

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Objective
From this dataset, I would like to figure out the kind of house type which has high price.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Dataset Identification

In [ ]:
df = pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/train.csv")

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.shape

Dataset df which is dataset train has total 1460 rows and 81 columns.

In [ ]:
numeric_columns = df.select_dtypes(exclude = ['object'])
numeric_columns

In [ ]:
numeric_columns.columns

In [ ]:
len(numeric_columns.columns)

Dataset has 38 numeric columns

In [ ]:
categoric_columns = df.select_dtypes(include = ['object'])
categoric_columns

In [ ]:
categoric_columns.columns

In [ ]:
len(categoric_columns.columns)

Dataset df has 43 categorical columns

In [ ]:
categoric_columns.nunique()

In [ ]:
numeric_columns.nunique()

In [ ]:
df.isna().sum().sum()

Dataset df has a lot of missing values in its various columns

In [ ]:
categoric_columns.isna().sum()

There are several columns which have too much missing values with total amount most likely 50% even 90%.

In [ ]:
numeric_columns.isna().sum()

In [ ]:
df.duplicated().sum()

In [ ]:
numeric_columns.describe().T

In [ ]:
categoric_columns.describe().T

In [ ]:
df.corr().T

In [ ]:
plt.figure(figsize=(20, 20))
sns.heatmap(df.corr(),
			cmap = 'BrBG',
			fmt = '.2f',
			linewidths = 2,
			annot = True)

# Data Preprocessing

Delete columns where have too much unique value.

In [ ]:
df_clean = df.drop(['Id'], axis=1)

In [ ]:
list_of_numeric = df_clean.select_dtypes(exclude = ['object']).columns.tolist()
list_of_numeric

In [ ]:
list_of_categoric = df_clean.select_dtypes(include = ['object']).columns.tolist()
list_of_categoric

In [ ]:
#numeric_columns = df_clean.select_dtypes(exclude = ['object'])
#categoric_columns = df_clean.select_dtypes(include = ['object'])

Filling missing values

In [ ]:
df_clean[list_of_numeric].isna().sum()

In [ ]:
df_clean[list_of_numeric].isna().any()

In [ ]:
for i in list_of_numeric:
    if df_clean[i].isna().any() == True:
        df_clean[i] = df_clean[i].fillna(df_clean[i].mean())
        print('Is {} has missing value? {}'.format(i, df_clean[i].isna().any()))
        
df_clean[list_of_numeric].isna().any()

In [ ]:
#df_clean['LotFrontage'] = df_clean['LotFrontage'].fillna(df_clean['LotFrontage'].mean())
#df_clean['MasVnrArea'] = df_clean['MasVnrArea'].fillna(df_clean['MasVnrArea'].mean())
#df_clean['GarageYrBlt'] = df_clean['GarageYrBlt'].fillna(df_clean['GarageYrBlt'].mean())

#df_clean[list_of_numeric].isna().any()

In [ ]:
df_clean[list_of_categoric].isna().sum()

In [ ]:
df_clean[list_of_categoric].isna().any()

Remove columns which have too many missing values (likely 50% to above) and fill missing values on other columns with mode.

In [ ]:
for i in list_of_categoric:
    if df_clean[i].isna().sum() > 600:
        del df_clean[i]
    else:
        df_clean[i] = df_clean[i].fillna(df_clean[i].mode()[0])
        print('Is {} has missing value? {}'.format(i, df_clean[i].isna().any()))

In [ ]:
list_of_categoric = df_clean.select_dtypes(include = 'object').columns.tolist()
list_of_categoric

In [ ]:
df_clean[list_of_categoric].isna().any()

In [ ]:
df_clean.isna().sum().sum()

In [ ]:
df_clean.shape

Normalization numeric columns in dataset df_clean.

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

In [ ]:
df_clean[list_of_numeric] = scaler.fit_transform(df_clean[list_of_numeric])

In [ ]:
df_clean.describe().T

Encoding categorical columns

In [ ]:
df_clean = pd.get_dummies(df_clean)

In [ ]:
df_clean

In [ ]:
#from sklearn.preprocessing import LabelEncoder

In [ ]:
#len(list_of_categoric)

In [ ]:
#for col in list_of_categoric:
#    df_clean[col] = LabelEncoder().fit_transform(df_clean[col]) 

In [ ]:
#df_clean[list_of_categoric]

In [ ]:
#df_clean

# Train Test Split

In [ ]:
from sklearn.model_selection import train_test_split

X = df_clean.drop(['SalePrice'], axis=1)
y = df_clean['SalePrice']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [ ]:
print("Dataset train contains:", X_train.shape[0], "rows and ", X_train.shape[1], "columns")
print("Dataset test contains:", X_test.shape[0], "rows and ", X_test.shape[1], "columns")

print("Variable target to train contains:", y_train.shape[0], "rows")
print("Variable target to test contains:", y_test.shape[0], "rows")

# Modeling with PCA

In [ ]:
#Here I decompose each row into 10 principal components
from sklearn.decomposition import PCA

def pca_dec(data, n):
  pca = PCA(n)
  X_dec = pca.fit_transform(data)
  return X_dec, pca

#Decomposing the train set:
pca_train_results, pca_train = pca_dec(X_train, 10)

#Decomposing the test set:
pca_test_results, pca_test = pca_dec(X_test, 10)

#Creating a table with the explained variance ratio
names_pcas = [f"PCA Component {i}" for i in range(1, 11, 1)]
scree = pd.DataFrame(list(zip(names_pcas, pca_train.explained_variance_ratio_)), columns=["Component", "Explained Variance Ratio"])
print(scree)

In [ ]:
#Sorting the values of the first principal component by how large each one is
df2 = pd.DataFrame({'PCA':pca_train.components_[0], 'Variable Names':list(X_train.columns)})
df2 = df2.sort_values('PCA', ascending=False)

#Sorting the absolute values of the first principal component by magnitude
df3 = pd.DataFrame(df2)
df3['PCA']=df3['PCA'].apply(np.absolute)
df3 = df3.sort_values('PCA', ascending=False)
#print(df2['Variable Names'][0:11])

df2.head()

What is conclusion? I dunno and I'll learn it more to know the answer!